## Issues

- I think the system is not consistently applying the variable names. 
- Need to set up a dev dataset 
- type matching
- need to revisit the speech simplifier -- seems to be oversimplifying.
    - "pick up any other ball" --> this specifically excludes currently activated/in-focus ball from consideration
- Need to figure out the overall robot algorithm here. 


## Features
1. Parsing 
    - Concept learning 
2. Rule induction
    - Commonsense 
3. Exception Induction
4. Interactive Dialog
    - Rule understanding (are the exceptions correct? is the rule correct?)
    - Rule Relaxation/Conflict Resolution (selecting which rule to relax)

# Algorithm 

## System level

### func learn(utterance U, robot action domain D, robot sensory domain S, ):
- robot asked to perform a task (action or goal instruction)
- robot interrupted 
- robot provided some fact about why it was interrupted. 
    if no fact, then robot asks "why"
- robot learns fact 

### func induce rule()
- robot induces a rule and generates likely exceptions
- robot refines rule with human interaction


### func refine()
- robot has a rule and exceptions and wants to check if it is correct
- human might be able to tell them if they 
    - missing some predicate
    - have an extra or incorrect predicate 
    - have any exceptions
        - robot can check if it already has exceptions. 
- repeat until human is happy. 

Functionality needed for refinement is -- ensuring that each of the RHS stuff in the rule is a tethered concept -- i.e., there is a proof for it. 



In [1]:
actions = [
    {'action': 'pickup',
    'description': 'Picks up items',
    'parameters': ['object'], 
    },
    {'action': 'putdown',
    'description': 'Puts down items',
    'parameters': ['object'], 
    },
    {'action': 'stack',
     'description': 'Stacks one object on top of another object',
     'parameters': ['object', 'object']
    }
]

consultants = [
    {'consultant': 'vision',
    'concepts': [
        {'name': 'clear',
         'description': 'Checks if an object has nothing on top of it',
         'parameters': ['object']
        },
        {'name': 'holding',
         'description': 'Checks if the agent is holding the object',
         'parameters': ['object']
        },
        {'name': 'on',
         'description': 'Checks if an object is top of another object',
        'parameters': ['object', 'object']},
        {'name': 'box',
         'description': 'Checks if an object is a box',
         'parameters': ['object']
        }
    ]
    }
]

In [2]:
# Given a list of dictionaries, and a key, return the entry in the list that matches
def find_dict_in_list(lst, key, target):
    for item in lst:
        if not key in item:
            #print("Key not in Dict")
            return None
        if item[key] == target:
            return item
    #print("Nothing found")
    return None

In [3]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [4]:
#llm = ChatOpenAI(model_name="gpt-4", temperature=0.0)
llm = OpenAI(temperature=0.0)

## Chains

### Speech Acts

A `speech act` is the action being performed by the utterance. We have several `types` of speech acts including `INSTRUCT`, `STATEMENT` etc. 

Note, we are talking about the nature of the utterance at the surface level. This does not distinguish between direct and indirect speech acts

In [5]:
## (0) Speech simplifier

template_simplifier= """
Simplify this sentence (don't get rid of determiners and articles):
\n
sentence:\n{utterance}\n
simplified:
"""
prompt_simplifier = PromptTemplate(
    input_variables=["utterance"],
    template=template_simplifier
)

chain_simplifier = LLMChain(llm=llm, prompt=prompt_simplifier)


In [6]:
## (1) Speech Act Classification 

template_speech_act_classifier= """
Decide whether the utterance below from a speaker to a listener is one of INSTRUCT, STATEMENT, GREETING, QUESTIONWH ('wh', questions), QUESTIONYN ('yes/no' questions), ACK (e.g. "yes" or "ok"), or UNKNOWN 
An INSTRUCT is an imperative statement or a request by the speaker to have the listener do an action or stop doing an action.
A QUESTIONWH is a 'wh' query (what, why, when, where, who) or request from a speaker for more information from the listener about the listeners knowledge, beliefs or perceptions
A QUESTIONYN is a 'yes/no' query or request from a speaker for more information from the listener about the listeners knowledge, beliefs or perceptions, but the speaker expects a yes or no for an answer
A STATEMENT is a statement of fact or opinion that the speaker conveys to a listener. 
A GREETING is an expression of social connection establishing the start of the conversation. E.g., "Hello"
A ACK is an acknowledgement (either "yes" or "no").
A UNKNOWN is an utterance not one of the above. 

utterance: \n{utterance}\n
act:
"""

prompt_speech_act_classifier = PromptTemplate(
    input_variables=["utterance"],
    template=template_speech_act_classifier
)

chain_speech_act_classifier = LLMChain(llm=llm, prompt=prompt_speech_act_classifier)


### Principle Intended Effect

The `principle intended effect` or `pie`, is what the speaker intends for the listener to do with the communicated utterance. This is a pragmatics issue and as such needs a mapping from the `speech act type` to the `pie`. Examples of `pie` include things like `want` and `wantBel` (where the speaker wants the listener to perform an action or speaker wants the listener to come believe something). Note, this is where reasoning about the indirect speech act comes into play. A `speech act type` of `question` (e.g., can you pass the salt) might result in the `pie` being `want` as opposed to `tellAnswer`. 

In [7]:
# Node code yet for PIE -- we can just have pragmatics handle that. 

### Actions and Concepts

#### Actions

`Actions` are symbolically specified functions that can be executed by the agent. Each `action` has an associated `action signature` which in turn has a `name`, `parameters` and `description`. For example: 

```
Action Definition:
    {'name': 'stack',
     'description': 'Stacks one object on top of another object',
     'parameters': ['object', 'object']
    }
```

The `name` is simply a string without spaces or special characters. The `description` is mpre general string containing an NL description of what the action is about. The `parameters` is an ordered list of strings. Each item in the list is a `parameter type symbol` that specifies the `type` that a `variable symbol` and also an `object constant symbol` can take. Note, this is just how actions are defined. Grounded actions have `parameters` that are constant symbols. See below. 

#### Concepts

`Concepts` are symbolically specified predicates that can be explicitly tracked by the agent. Each `concept` also has an associated `concept signature` which in turn has a `name`, `parameters` and `description`. For example: 

```
Concept:
    {'name': 'on',
     'description': 'Checks if an object is top of another object',
     'parameters': ['object', 'object']
    }
```

#### Grounded vs Lifted

A `grounded action` is an action in which all the parameters are replaced with specific `object constant symbols`. Similarly a `grounded concept` is a concept in which all parameters are replaced with specific `object constant symbols`. Note, for a grounded action or concept to be type-matched, the `object constant symbols` must have a `type` that matches the parameter `type` specified in the action/concept definition. 

A `lifted action/concept` is an action/concept in which one or more parameters are `variable symbols`. Here too, the parameters MUST be type-matched, i.e., any variable or object `constant symbols` must have a `type` that matches the parameter `type` specified in the action/concept definition. 

#### Tethered Action/Concept

A `tethered action` is an action that the robot has the ability to execute. A `tethered concept` is a concept that the robot can perceive its truth value from the current state. 

#### Novel Action/Concept 

A `novel action or concept` is an action or concept that the robot does not already knowledge of. It is not present in the agent's action database or set of conceptual beliefs or consultant capabilities.

**Note:**: One could think about derived predicates, i.e., concepts that are _only_ defined as the head of horn clauses in Belief. One could argue that these are ungrounded in a way, but they are tethered because they can be infered. But, I am arguing here is that these are defined in terms  of other concepts and as long as they are tethered to a grounded concept, we should be okay. 


Let's walk through an example:
```
    Utterance: Evan owns the purple box. 

    Concepts: 
    own(obj35, evan), purple(obj35), box(obj35), agent(evan)
    
```
Let's say the robot has an implemented "purple" detector and a "box" detector. It also knows that evan is an agent. 

Let's say the robot has no prior knowledge of "own(X)", which means it is a new concept. "own(X)" is an untethered concept that is also novel. When asserted by the human, the robot will come to believe that "own(obj35, evan)", but it cannot infer whether another object is owned by evan or other objects are owned by others.

We say here that "own(X)" is `untethered` but that doesn't stop the robot from reasoning about it. Crucially, if it has another rule: 

```
constraintViolation(O) :- own(O,A), holding(O), agent(A), object(O), A!=self. 
```
With this rule and the above fact, the agent can reason that they have violated a constraint, without really understanding what "ownership" actually means. 


### Propositional Content (PC)

#### Core Propositional Content (CPC)
The `core propositional content` or `cpc` has to do with the "thing" that the speaker is talking about. Searle calls this the propositional content condition. For example, "The purple box is open" has a `cpc` of "isOpen(X)". "Open the purple box" has a `cpc` of "open(X)". Note here that "open" is an action whereas "isOpen" is a concept. We will talk about actions and concepts later. 

#### Supporting Propositional Content (SPC) 
The `supporting propositional content` or `spc` has to do with constraining information about the `cpc`. So, "The purple box is open" has `spc` of {purple(X), box(X), DEFINITE(X)}. Similarly, "Open the purple box" has an `spc` of {purple(X), box(X), DEFINITE(X)}. Note the `spc` is the same in both cases. Note, also that the `spc` is a collection of concepts. But it could be more complicated like "Open the purple box you had opened and closed yesterday". Again, the `cpc` is still "open(X)", but now the `spc` is more complex {purple(X), box(X), DEFINITE(X), ...history(X,\[open(X), close(X)\])...}

Let's start with the `cpc`: 


In [8]:
# CPC determination

## (2a) CPC associated with concepts because we have a STATEMENT Speech act. 

template_cpc_concept_selector="""
Generate a single string (without spaces or other special characters) that represents the core propositional content of the utterance. 
Use the following procedure:
1. Select the core portion of the utterance that represents the property of an object or concept associated with the object that the speaker is telling the listener that the listener does not already know.
2. Then, compare the name and description of each of the properties below to the utterance. Narrow the list of properties to include only those with a semantically similar name or description to the core portion of the utterance. 
3. If the narrowed list contains one property, then return its name. If it contains no properties, then generate a new symbol -- a short (4-5 character) string to represent this portion/property in the utterance. If it contains more than one property, then return AMBIGUOUS.

\n\n LIST OF AVAILABLE PROPERTIES/CONCEPTS \n:
{concepts}

utterance: \n{utterance}\n
core proposition name:
"""


prompt_cpc_concept_selector = PromptTemplate(
    input_variables=["utterance", "concepts"],
    template=template_cpc_concept_selector
)

chain_cpc_concept_selector = LLMChain(llm=llm, prompt=prompt_cpc_concept_selector)

##---------
# (2b) CPC associated with Actions because we have an INSTRUCT action

template_cpc_action_selector= """
Select an action from the list of available actions that is most relevant to the given utterance. 
To decide the applicable action, use the following procedure to systematically filter the most relevant action:
1. Check the dialog_type. Narrow the list of actions to only include those actions that are ontic actions or world-modifying actions. 

2. Then, compare the name and description of the action to the utterance. Narrow the list of actions to include only those with a semantically similar name or description to the utterance. 

3. If the narrowed list contains one action, then return its name. If it contains no actions, then return NONE. If it contains more than one action, then return AMBIGUOUS.


\n\n LIST OF AVAILABLE ACTIONS \n:
{actions}

utterance: \n{utterance}\n
action:
"""

prompt_cpc_action_selector = PromptTemplate(
    input_variables=["utterance", "actions"],
    template=template_cpc_action_selector
)

chain_cpc_action_selector = LLMChain(llm=llm, prompt=prompt_cpc_action_selector)


In [9]:
# SPC determiner 

## (4) Identify a referrent object and its properties. 
### Given an utterance, an action or a property (the core proposition), and its parameter types, return a set of properties 

template_referent_properties= """
Identify descriptors in the utterance that refer to an entity which is an argument or parameter in the core proposition of the utterance.
Use the following procedure:
1. For each of the parameter types, identify what descriptive terms refer to the parameter type from the utterance. 
That is think of an entity is being referred to by the parameter types, and see what descriptors refer to this entity.
2. Build a predicates for each descriptive terms. The predicates have a functor name that is the descriptor and a variable name.
Hypothesize variable names (e.g., VAR0, VAR1, etc.). The predicates should be of the form `descriptive term(variable names)`
3. Compose the predicates into a list 
4. Return the list as properties. 

Remember, these are descriptive properties (adjectives in a sense) and do NOT include cues for adverbs, articles, determiners, pronouns etc.
Also remember that functors cannot contain spaces, so replace spaces with underscores.
Also, remember the output list of properties should not include the core proposition itself. 
\n\nEXAMPLE\n
utterance: Pick up the blue ball
core proposition: pickup
parameter types: ["object"]
properties: ["blue(VAR0)", "ball(VAR0)"]
\n\n
\n\nEXAMPLE\n
utterance: Stack the cup on any other cup
core proposition: stack
parameter types: ["object","object"]
properties: ["cup(VAR0)", "cup(VAR1)"]
\n\n
utterance: \n{utterance}\n
core proposition: \n{cpc}\n
parameter types: \n{parameters}\n
properties:
"""
prompt_referent_properties = PromptTemplate(
    input_variables=["utterance", "cpc", "parameters"],
    template=template_referent_properties
)

chain_referent_properties = LLMChain(llm=llm, prompt=prompt_referent_properties)



## (5) Ref Res: For each variable determine it's cognitive status 
### For each variable we are figuring out the one or more givenness hierarchy statuses

template_cognitive_status= """
For each variable in the properties, decide which ONE (and only one) of the following five cognitive statuses the variables in the below properties could fall into:
statuses: [INFOCUS, ACTIVATED", FAMILIAR, DEFINITE, INDEFINITE]

As shown in the table below, the Givenness Hierarchy is comprised of six hierarchically nested tiers of cognitive status, 
where information with one cognitive status can be inferred to also have all
lower statuses. Each level of the GH is “cued” by a set
of linguistic forms, as seen in the table. For example, the second
row of the table shows that the definite use of “this” can be
used to infer that the speaker assumes the referent to be at
least activated to their interlocutor.
\n\n
Cognitive Status | Mnemonic Status | Form |
-----------------|-----------------|------|
INFOCUS | in the focus of attention | it |
ACTIVATED | in short term memory | this,that,this N |
FAMILIAR | in long term memory| that N |
DEFINITE | in long term memory  or new | the N |
INDEFINITE | new or hypothetical | a N |
\n\n

When deciding the one cognitive status for each variable, use the table above and compare the form (pronoun, determiner, article) of the utterance to its status.
\n\nExample:\n
utterance: Pick up the blue ball
properties: ["blue(VAR0)", "ball(VAR0)"]
cognitive status: ["DEFINITE(VAR0)"]
\n\n
utterance: \n{utterance}\n
properties: \n{properties}\n
cognitive status:
"""



prompt_cognitive_status = PromptTemplate(
    input_variables=["utterance", "properties"],
    template=template_cognitive_status
)

chain_cognitive_status = LLMChain(llm=llm, prompt=prompt_cognitive_status)



In [10]:
import ast
import re

def parse_utterance(utterance, actions, properties):
    #print(f"Utterance: {utterance}")
    #print(f"Actions: {actions}")
    # utterance = chain_simplifier.run(utterance=utterance)
    speech_act = chain_speech_act_classifier.run(utterance=utterance)
    if speech_act == 'INSTRUCT':
        cpc = chain_cpc_action_selector.run(utterance=utterance, actions=actions)
    elif speech_act == 'STATEMENT':
        cpc = chain_cpc_concept_selector.run(utterance=utterance,concepts=consultants)
    else:
        return {} 
        
    parameters = []
    properties = []
    cognitive_status = []
    variables = []
    if not cpc=="NONE" and not cpc=="AMBIGUOUS":
        # Get params
        if speech_act == 'INSTRUCT':
            relevant_cpc = find_dict_in_list(actions, 'action', cpc)
        elif speech_act == 'STATEMENT':
            print(cpc)
            print(consultants[0]['concepts'])
            relevant_cpc = find_dict_in_list(consultants[0]['concepts'], 'name', cpc)
        else: 
            return {}
        parameters = relevant_cpc['parameters']
        
        # Get properties
        properties = chain_referent_properties.run(utterance=utterance, cpc=cpc, parameters=parameters)
        properties = ast.literal_eval(properties)
        
        # Get list of variable names
        def get_args(predicate):
            return re.search('\(([^)]+)', predicate).group(1)
        
        if properties:
            variables = list(set([get_args(x) for x in properties]))
        
        
        cognitive_status = chain_cognitive_status.run(utterance=utterance, properties=properties)
        
        #eval cog status
        cognitive_status = ast.literal_eval(cognitive_status)
        
    output = {'utterance': utterance,
              'speech_act':speech_act,
              'cpc': cpc,
             'parameters': parameters,
             'properties': properties,
              'variables': variables,
             'cognitive_status': cognitive_status}
    return output



def parse(speaker, listener, utterance, actions, consultants):
    output = parse_utterance(utterance, actions, consultants)
    variables = ",".join(output['variables'])
    properties = ",".join(output['properties'])
    cognitive_status = ",".join(output['cognitive_status'])
    cpc = output['cpc']
    speech_act = output['speech_act']
    
    template = "{speech_act}({speaker},{listener},{cpc}({variables}),{{{properties},{cognitive_status}}})"
    parsed = template.format(speech_act=speech_act,
                            speaker=speaker,
                            listener=listener,
                            cpc=cpc,
                            variables=variables,
                            properties=properties,
                            cognitive_status=cognitive_status)
        
    return parsed, output

In [ ]:
import gradio as gr

def nlu(utterance):
    parsed, output = parse("brad", "self", utterance, actions, consultants)
    return parsed, output

demo = gr.Interface(fn=nlu, inputs="text", outputs=["text", "text"])

demo.launch(debug=True) 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


belong
[{'name': 'clear', 'description': 'Checks if an object has nothing on top of it', 'parameters': ['object']}, {'name': 'holding', 'description': 'Checks if the agent is holding the object', 'parameters': ['object']}, {'name': 'on', 'description': 'Checks if an object is top of another object', 'parameters': ['object', 'object']}, {'name': 'box', 'description': 'Checks if an object is a box', 'parameters': ['object']}]


Traceback (most recent call last):
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/gradio/routes.py", line 414, in run_predict
    output = await app.get_blocks().process_api(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1320, in process_api
    result = await self.call_function(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1048, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await futur

belong
[{'name': 'clear', 'description': 'Checks if an object has nothing on top of it', 'parameters': ['object']}, {'name': 'holding', 'description': 'Checks if the agent is holding the object', 'parameters': ['object']}, {'name': 'on', 'description': 'Checks if an object is top of another object', 'parameters': ['object', 'object']}, {'name': 'box', 'description': 'Checks if an object is a box', 'parameters': ['object']}]


Traceback (most recent call last):
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/gradio/routes.py", line 414, in run_predict
    output = await app.get_blocks().process_api(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1320, in process_api
    result = await self.call_function(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/gradio/blocks.py", line 1048, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/home/vsarathy/.cache/pypoetry/virtualenvs/gsp-KcVP1gnp-py3.10/lib/python3.10/site-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await futur

## Run a short dev test

In [14]:
import json

data = [{"utterance": "Pick up the ball", "act": "INSTRUCT"},
        {"utterance": "Get the ball", "act": "INSTRUCT"},
        {"utterance": "Pick up a ball", "act": "INSTRUCT"},
        {"utterance": "Pick up this blue ball", "act": "INSTRUCT"},
        {"utterance": "Put the ball down on the table", "act":"INSTRUCT"},
       {"utterance": "Get the circuit breaker on the work area", "act": "INSTRUCT"},
       {"utterance": "First raise your arms", "act": "INSTRUCT"},
       {"utterance": "Dempster, who do you trust?", "act": "QUESTION"},
       {"utterance": "Dempster, do you see an object?", "act": "QUESTION"},
       {"utterance": "The area behind you is safe", "act": "STATEMENT"},
       {"utterance": "The object in front of you is a ball", "act": "STATEMENT"},
       {"utterance": "It uses a medical caddy", "act": "STATEMENT"}]

for item in data:
    parsed, output = parse("brad", "self", item['utterance'], actions, consultants)
    print(json.dumps(output, indent=2))   
    print("PARSED: ", parsed,"\n")

{
  "utterance": "Pick up the ball",
  "speech_act": "INSTRUCT",
  "cpc": "pickup",
  "parameters": [
    "object"
  ],
  "properties": [
    "ball(VAR0)"
  ],
  "variables": [
    "VAR0"
  ],
  "cognitive_status": [
    "DEFINITE(VAR0)"
  ]
}
PARSED:  INSTRUCT(brad,self,pickup(VAR0),{ball(VAR0),DEFINITE(VAR0)}) 

{
  "utterance": "Get the ball",
  "speech_act": "INSTRUCT",
  "cpc": "NONE",
  "parameters": [],
  "properties": [],
  "variables": [],
  "cognitive_status": []
}
PARSED:  INSTRUCT(brad,self,NONE(),{,}) 

{
  "utterance": "Pick up a ball",
  "speech_act": "INSTRUCT",
  "cpc": "pickup",
  "parameters": [
    "object"
  ],
  "properties": [
    "ball(VAR0)"
  ],
  "variables": [
    "VAR0"
  ],
  "cognitive_status": [
    "INDEFINITE(VAR0)"
  ]
}
PARSED:  INSTRUCT(brad,self,pickup(VAR0),{ball(VAR0),INDEFINITE(VAR0)}) 



Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


{
  "utterance": "Pick up this blue ball",
  "speech_act": "INSTRUCT",
  "cpc": "pickup",
  "parameters": [
    "object"
  ],
  "properties": [
    "this(VAR0)",
    "blue(VAR0)",
    "ball(VAR0)"
  ],
  "variables": [
    "VAR0"
  ],
  "cognitive_status": [
    "ACTIVATED(VAR0)"
  ]
}
PARSED:  INSTRUCT(brad,self,pickup(VAR0),{this(VAR0),blue(VAR0),ball(VAR0),ACTIVATED(VAR0)}) 



Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


{
  "utterance": "Put the ball down on the table",
  "speech_act": "INSTRUCT",
  "cpc": "putdown",
  "parameters": [
    "object"
  ],
  "properties": [
    "ball(VAR0)",
    "table(VAR0)"
  ],
  "variables": [
    "VAR0"
  ],
  "cognitive_status": [
    "DEFINITE(VAR0)"
  ]
}
PARSED:  INSTRUCT(brad,self,putdown(VAR0),{ball(VAR0),table(VAR0),DEFINITE(VAR0)}) 

{
  "utterance": "Get the circuit breaker on the work area",
  "speech_act": "INSTRUCT",
  "cpc": "NONE",
  "parameters": [],
  "properties": [],
  "variables": [],
  "cognitive_status": []
}
PARSED:  INSTRUCT(brad,self,NONE(),{,}) 



Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


{
  "utterance": "First raise your arms",
  "speech_act": "INSTRUCT",
  "cpc": "NONE",
  "parameters": [],
  "properties": [],
  "variables": [],
  "cognitive_status": []
}
PARSED:  INSTRUCT(brad,self,NONE(),{,}) 



KeyError: 'variables'

# DO NOT USE BELOW THIS LINE

In [ ]:
# Spot testing
data = [{"utterance": "Pick up the ball", "act": "INSTRUCT"},
        {"utterance": "Get the ball", "act": "INSTRUCT"},
        {"utterance": "Pick up a ball", "act": "INSTRUCT"},
        {"utterance": "Pick up this blue ball", "act": "INSTRUCT"},
        {"utterance": "Put the ball down on the table", "act":"INSTRUCT"},
       {"utterance": "Get the circuit breaker on the work area", "act": "INSTRUCT"},
       {"utterance": "First raise your arms", "act": "INSTRUCT"},
       {"utterance": "Dempster, who do you trust?", "act": "QUESTION"},
       {"utterance": "Dempster, do you see an object?", "act": "QUESTION"},
       {"utterance": "The area behind you is safe", "act": "STATEMENT"},
       {"utterance": "The object in front of you is a ball", "act": "STATEMENT"},
       {"utterance": "It uses a medical caddy", "act": "STATEMENT"}]

In [ ]:

## Explanations
### Given a set of instructions that produced a parse, generate an argument for the parse is correct. 

template_argument = """
Generate an argume
Cognitive Status | Mnemonic Status | Form |
-----------------|-----------------|------|
INFOCUS | in the focus of attention | it |
ACTIVATED | in short term memory | this,that,this N |
FAMILIAR | in long term memory| that N |
UNIQUELY IDENTIFIABLE | in long term memory  or new | the N |
REFERENTIAL | nt for why the system came up with the particular parse of the utterance. 
To generate this argument ONLY use the below information, which contains prior prompts. 


\n\nACTION SELECTION PROMPT:\n
Select an action from the list of available actions that is most relevant to the given utterance. 
To decide the applicable action, use the following procedconsultantsure to systematically filter the most relevant action:
1. Check the dialog_type. If it is an INSTRUCT, then narrow the list of actions to only include those actions that are ontic actions or world-modifying actions. If the utterance is a QUESTION, then narrow the list of actions to only include those actions that are querying actions. If the utterance is a STATEMENT, then narrow the list of actions to only include those actions that are epistemic actions, or actions that change the belief state of the listener. 

2. Then, compare the name and description of the action to the utterance. Narrow the list of actions to include only those with a semantically similar name or description to the utterance. 

3. If the narrowed list contains one action, then return its name. If it contains no actions, then return NONE. If it contains more than one action, then return AMBIGUOUS.

\n\n PROPERTIES EXTRACTION\n
Identify descriptors in the utterance that refer to an entity which is an argument or parameter in the action.
Use the following procedure:
1. For each of the parameter types, identify what descriptive terms refer to the parameter type from the utterance. 
That is think of an entity is being referred to by the parameter types, and see what descriptors refer to this entity.
2. Build a predicates for each descriptive terms. The predicates have a functor name that is the descriptor and a variable name.
Hypothesize variable names (e.g., VAR0, VAR1, etc.). The predicates should be of the form `descriptive term(variable names)`
3. Compose the predicates into a list 
4. Return the list as properties. 

Remember, these are descriptive properties (adjectives in a sense) and do not include cues for articles, determiners, pronouns etc.
Also remember that functors cannot contain spaces, so replace spaces with underscores.
\n\nEXAMPLE\n
utterance: Pick up the blue ball
action: pickup
parameter types: ["object"]
properties: ["blue(VAR0)", "ball(VAR0)"]
\n\n

\n\n IDENTIFYING COGNITIVE STATUS OF ENTITIES IN REFERRING EXPRESSIONS\n
For each variable in the properties, decide which ONE of the following five cognitive statuses the variables in the below properties could fall into:
statuses: [INFOCUS, ACTIVATED", FAMILIAR, UINIQUELY_IDENTIFIABLE, REFERENTIAL, TYPE_IDENTIFIABLE]

As shown in the table below, the Givenness Hierarchy is comprised of six hierarchically nested tiers of cognitive status, 
where information with one cognitive status can be inferred to also have all
lower statuses. Each level of the GH is “cued” by a set
of linguistic forms, as seen in the table. For example, the second
row of the table shows that the definite use of “this” can be
used to infer that the speaker assumes the referent to be at
least activated to their interlocutor.
\n\n
Cognitive Status | Mnemonic Status | Form |
-----------------|-----------------|------|
INFOCUS | in the focus of attention | it |
ACTIVATED | in short term memory | this,that,this N |
FAMILIAR | in long term memory| that N |
UNIQUELY IDENTIFIABLE | in long term memory  or new | the N |
REFERENTIAL | new or hypothetical|  indefinite this N |
TYPE IDENTIFIABLE | new or hypothetical | a N |

Remember, we need an argument that argues in favor of the parse, but we need to talk about why parse may or may not work too. 

utterance: \n{utterance}\n
parse: \n{parse}\n
argument:
"""

prompt_argument = PromptTemplate(
    input_variables=["utterance", "parse"],
    template=template_argument
)

chain_argument = LLMChain(llm=llm, prompt=prompt_argument)


In [ ]:
import json

for item in data:
    parsed, output = parse("brad", "self", item['utterance'], actions)
    print(json.dumps(output, indent=2))   
    print("PARSED: ", parsed,"\n")

In [ ]:
def explain(utterance, parse):
    # Generate explanation
    print("Generating explanation")
    argument = chain_argument.run(utterance=utterance, parse=parse)
    return argument


In [ ]:
# Spot test

utterance01 = "pick up the ball"
parsed01 = parse("vasanth", "self", utterance01, actions)
explanation01 = explain(utterance01, parsed01)
print(f"Utterance: {utterance01}\nParse: {parsed01[0]}\n\nExplanation: {explanation01}")

# Research problems


1. Incomplete action imperatives ("Stack the block" ... on top of what?) 
2. No matching action
3. No matching property
4. 

# v0.1

The goal of this is to get a basic version up and running

Approach 1
- Infer dialog act
- Look only at action names at first (maybe descriptions), if ambiguous, then look at action signatures, if ambiguous look at pre/eff, if still ambiguous ask for clarification. 
    - Need to be more general here. The imperative could be specifying a sequence of actions (not sure DIARC can handle this anyway), or an action that cannot occur unless another action is performed first --> constant interaction with planner is needed. Sometimes action imperatives are even just goal directives in disguise (go to the door --> no goToDoor action, but at(door) is a possible state. How to convert "go to" to "be at". "put two blue blocks on top of the red one". 


idea
- Check if action names exist, if not then reword the utterance as a goal, and see if it can be handled that way
- If only one name exists, then check action signature and types
    - if that works, then check if precons are met, and if so, perform action
    - If not, then set precons as goal, plan and generate action sequence, and then also perform target action
- If multiple matches, then check to see the "best" action signature
    - Select best action signature by 

Other things:
- Get a pddl-based planner involved --> external calls 

Variations
- human involvement could include providing some set of conditions that specify several states, or they provide a a group of actions (a miniplan) and refer to that by a name
- human could provide action performance constraints - do X, but don't touch Y.

- Challenges with reference resolution .
